In [1]:
import numpy as np
from astropy.coordinates import SkyCoord
from astropy import units as u
from time import time

# Crossmatching : optimised using Astropy Module

In [2]:
def crossmatch(coords1, coords2, m):
    start_time = time()
    max_radius = m
    matches = []
    no_matches = []
    
    # Convert to astropy coordinate objects
    coords1_sc = SkyCoord(coords1*u.degree, frame = 'icrs')
    coords2_sc = SkyCoord(coords2*u.degree, frame = 'icrs')
    
    # Crossmatching
    closest_ids, closest_dists, _ = coords1_sc.match_to_catalog_sky(coords2_sc)
    
    for id1, (closest_id2, dist) in enumerate(zip(closest_ids, closest_dists)):
        closest_dist = dist.value
        if closest_dist > max_radius:
            no_matches.append(id1)
        else:
            matches.append([id1, closest_id2, closest_dist])
    
    time_taken = time() - start_time
    return matches, no_matches, time_taken

In [3]:
def hms2dec(h, m, s):
    dec = h + m/60 + s/3600
    return dec*15

In [4]:
def dms2dec(d, m, s):
    sign = d/abs(d)
    dec = abs(d) + m/60 + s/3600
    return sign*dec

In [11]:
def import_bss():
    res = []
    data = np.loadtxt('bss.dat', usecols = range(1,7))
    for i, row in enumerate(data, 1):
        res.append((hms2dec(row[0], row[1], row[2]), dms2dec(row[3], row[4], row[5])))
    return res

In [12]:
def import_super():
    data = np.loadtxt('super.csv', delimiter = ',', skiprows = 1, usecols = [0,1])
    res = []
    for i, row in enumerate(data, 1):
        res.append((row[0], row[1]))
    return res

In [13]:
bss_cat = import_bss()
super_cat = import_super()

In [14]:
bss_cat[:3]

[(1.1485416666666666, -47.60530555555556),
 (2.6496666666666666, -30.463416666666667),
 (2.7552916666666665, -26.209194444444442)]

In [16]:
super_cat[:3]

[(0.0603176, -85.6561327), (1.1485082, -47.6054131), (1.2794331, -1.5459014)]

In [17]:
matches, no_matches, time_taken = crossmatch(bss_cat, super_cat, 5)

In [18]:
print('matches: ', matches)

matches:  [[0, 1, 0.00010988610938462042], [1, 3, 0.0007649845967254174], [2, 4, 0.00020863352870693826], [3, 5, 0.00012867299967284906], [4, 10, 2.9946594679884266], [6, 10, 7.666235090274782e-05], [7, 13, 0.00020833046102231966], [8, 14, 0.0001774015026066542], [9, 16, 0.0001607960443969137], [10, 27, 2.2007192949063925], [11, 17, 0.00034939446067687896], [12, 21, 0.0001217054159754761], [13, 23, 0.008314435904854872], [14, 24, 0.00021977416884395184], [15, 25, 0.00013282868901280515], [16, 26, 3.1815304279419186e-05], [17, 28, 0.00019843774684235777], [18, 29, 0.00011585178516820577], [19, 30, 0.0002627136922352513], [20, 31, 0.00039932330328367263], [21, 32, 7.365355772161354e-05], [22, 33, 0.00022280948961224468], [23, 34, 0.000154819605838957], [24, 36, 0.0003688060288873599], [25, 38, 0.0003750312712816945], [26, 40, 0.00011834492005111738], [27, 41, 0.0001221710406318912], [28, 41, 2.2854072340956555], [29, 42, 0.0005573032265468897], [30, 43, 0.00018231015875755786], [31, 45, 

In [19]:
print('unmatched: ', no_matches)

unmatched:  [5]


In [20]:
print('time: ', time_taken)

time:  0.010174274444580078
